<div style="background-color: #bfd630; font-family: Calibri, sans-serif; padding: 20px;">



   <div style="text-align: center;">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaJWG7PzF3toxaRMB1-JicpqMgJuEXATd0fg&" style="width: 120px; margin-top: 20px; margin-bottom: 60px;">
   </div>

   <div style="text-align: center; font-size: 24px; font-weight: bold; font-family: Calibri; color: #000000; margin-bottom: 10px;">
      Machine Learning Project | To Grant or Not to Grant
   </div>
   <div style="text-align: center; font-family: Calibri; font-size: 22px; color: #000000; font-weight: bold; margin-bottom: 20px;">
      3. Feature selection and Model building
   </div>

   <div style="text-align: center; font-size: 18px; font-family: Calibri; font-weight: bold; color: #333333; margin-bottom: 5px;">
      Nova Information Management School
   </div>

   <div style="text-align: center; font-size: 18px; font-family: Calibri; font-weight: bold; color: #333333; margin-bottom: 20px;">
      Universidade Nova de Lisboa
   </div>
        <div style="text-align: center; font-size: 16px; font-family: Calibri; font-weight: bold; color: #333333; margin-bottom: 10px;">
      Master in Data Science and Advanced Analytics
   </div>
 
   <div style="text-align: center;">
      <img src="https://cdn.prod.website-files.com/617accb8b04ef2b3feffa61b/6581e90d485a9976c3576a46_how-does-workers-comp-work.jpg" style="width: 350px; margin-top: 20px; margin-bottom: 60px;">
   </div>





   <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; font-weight: bold; margin-bottom: 20px;">
      Project Group: 32
   </div>

   <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; margin-bottom: 40px">
      Klimentina Gilevska -  20240747 <br>
      Maria Assunção -  20211605 <br>
      Rita Matos -  20211642 <br>
      Rita Wang -  20240551 <br>
      Sven Goerdes -  20240503
   </div>

   <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; margin-bottom: 10px">
      Fall/Spring Semester 2024-2025
   </div>

  <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; margin-bottom: 20px;">
      22nd of December 2024
   </div>

   
</div>


# 1. Importing Libraries & Dataset

## 1.1 Import Necessary Libraries

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Scaling
from sklearn.preprocessing import MinMaxScaler

 #Correlation Heatmap
from matplotlib.colors import LinearSegmentedColormap

#Statistical Test
from scipy import stats
from sklearn.impute import SimpleImputer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None)

import sys

sys.path.append('../pipeline_scripts')
from feature_selection_transformers import *

## 1.2 Load the Dataset

In [92]:
# You can download the data in the source that is linked above the table of contents

# Read in the data
X_train_encoded = pd.read_csv('../project_data/X_train_encoded.csv', delimiter=',', index_col=0)
X_val_encoded = pd.read_csv('../project_data/X_val_encoded.csv', delimiter=',', index_col=0)

y_train = pd.read_csv('../project_data/y_train.csv',delimiter=',', index_col=0)
y_val= pd.read_csv('../project_data/y_val.csv', delimiter=',', index_col=0)

X_test_encoded = pd.read_csv('../project_data/X_test_encoded.csv',index_col=0)

# 2. Prepare the Dataset

## 2.1 Encode the Target Variable

> We decided to label encode the target variable in the following way:
>    | Claim Injury Type   | Encoded Value |
>    |---------------------|---------------|
>    | 4. TEMPORARY        | 3             |
>    | 2. NON-COMP         | 1             |
>    | 5. PPD SCH LOSS     | 4             |
>    | 3. MED ONLY         | 2             |
>    | 6. PPD NSL          | 5             |
>    | 1. CANCELLED        | 0             |
>    | 8. DEATH            | 7             |
>    | 7. PTD              | 6             |

In [93]:
claim_injury_type_mapping = {
    '4. TEMPORARY': 3,
    '2. NON-COMP': 1,
    '5. PPD SCH LOSS': 4,
    '3. MED ONLY': 2,
    '6. PPD NSL': 5,
    '1. CANCELLED': 0,
    '8. DEATH':7,
    '7. PTD': 6
}

y_train_encoded = y_train['Claim Injury Type'].map(claim_injury_type_mapping)
y_val_encoded = y_val['Claim Injury Type'].map(claim_injury_type_mapping)

## 2.2 Non-encoded Target Variable as Series

> We convert the target variable to non-series 

In [94]:
y = y_train.squeeze()  # Converts a DataFrame with one column to a Series

## 2.2 Separate the Variables into Numerical and Categorical

> __Separate the variables into categorical and numerical__
>
> | Numerical Columns                                     | Categorical Columns                          |
> |-------------------------------------------------------|----------------------------------------------|
> | Age at Injury                                         | Alternative Dispute Resolution_binary        |
> | IME-4 Count                                           | COVID-19 Indicator_binary                   |
> | Number of Dependents                                  | Attorney/Representative_binary              |
> | Days_between_Assembly Date_Accident Date_log          | Carrier Type_Self-insured Private Entity     |
> | Days_between_C-2 Date_Imputed_Accident Date_log               | Carrier Type_Self-insured Public Entity      |
> | Average Weekly Wage_Imputed_log                       | Carrier Type_Special Funds                  |
> | Industry Code_encoded_5. PPD SCH LOSS                | Carrier Type_State Insurance Fund           |
> | Industry Code_encoded_2. NON-COMP                    | C-3 Date_nabinary                           |
> | Industry Code_encoded_3. MED ONLY                    | First Hearing Date_nabinary                |
> | Industry Code_encoded_4. TEMPORARY                   | Accident Date_Season_Spring                |
> | Industry Code_encoded_1. CANCELLED                   | Accident Date_Season_Summer                |
> | Industry Code_encoded_8. DEATH                       | Accident Date_Season_Winter                |
> | Industry Code_encoded_6. PPD NSL                     | Average Weekly Wage_nabinary               |
> | Industry Code_encoded_7. PTD                         |                                              |
> | WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS    |                                              |
> | WCIO Cause of Injury Code_encoded_2. NON-COMP        |                                              |
> | WCIO Cause of Injury Code_encoded_3. MED ONLY        |                                              |
> | WCIO Cause of Injury Code_encoded_4. TEMPORARY       |                                              |
> | WCIO Cause of Injury Code_encoded_1. CANCELLED       |                                              |
> | WCIO Cause of Injury Code_encoded_8. DEATH           |                                              |
> | WCIO Cause of Injury Code_encoded_6. PPD NSL         |                                              |
> | WCIO Cause of Injury Code_encoded_7. PTD             |                                              |
> | WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS   |                                              |
> | WCIO Nature of Injury Code_encoded_2. NON-COMP       |                                              |
> | WCIO Nature of Injury Code_encoded_3. MED ONLY       |                                              |
> | WCIO Nature of Injury Code_encoded_4. TEMPORARY      |                                              |
> | WCIO Nature of Injury Code_encoded_1. CANCELLED      |                                              |
> | WCIO Nature of Injury Code_encoded_8. DEATH          |                                              |
> | WCIO Nature of Injury Code_encoded_6. PPD NSL        |                                              |
> | WCIO Nature of Injury Code_encoded_7. PTD            |                                              |
> | WCIO Part Of Body Code_encoded_5. PPD SCH LOSS       |                                              |
> | WCIO Part Of Body Code_encoded_2. NON-COMP           |                                              |
> | WCIO Part Of Body Code_encoded_3. MED ONLY           |                                              |
> | WCIO Part Of Body Code_encoded_4. TEMPORARY          |                                              |
> | WCIO Part Of Body Code_encoded_1. CANCELLED          |                                              |
> | WCIO Part Of Body Code_encoded_8. DEATH              |                                              |
> | WCIO Part Of Body Code_encoded_6. PPD NSL            |                                              |
> | WCIO Part Of Body Code_encoded_7. PTD                |                                              |
> | Accident Datemonth                                   |                                              |
> | Industry Code_freq                                   |                                              |
> | WCIO Cause of Injury Code_freq                       |                                              |
> | WCIO Nature of Injury Code_freq                      |                                              |
> | WCIO Part Of Body Code_freq                          |                                              |
> | Carrier Type_freq                                    |                                              |
> | Carrier Name_freq                                    |                                              |



In [95]:
num_columns = ['Age at Injury', 'IME-4 Count', 'Number of Dependents',
               'Days_between_Assembly Date_Accident Date_log',
               'Days_between_C-2 Date_Imputed_Accident Date_log', 'Average Weekly Wage_Imputed_log',
               'Industry Code_encoded_5. PPD SCH LOSS',
               'Industry Code_encoded_2. NON-COMP',
               'Industry Code_encoded_3. MED ONLY',
               'Industry Code_encoded_4. TEMPORARY',
               'Industry Code_encoded_1. CANCELLED', 'Industry Code_encoded_8. DEATH',
               'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
               'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
               'WCIO Cause of Injury Code_encoded_2. NON-COMP',
               'WCIO Cause of Injury Code_encoded_3. MED ONLY',
               'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
               'WCIO Cause of Injury Code_encoded_1. CANCELLED',
               'WCIO Cause of Injury Code_encoded_8. DEATH',
               'WCIO Cause of Injury Code_encoded_6. PPD NSL',
               'WCIO Cause of Injury Code_encoded_7. PTD',
               'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
               'WCIO Nature of Injury Code_encoded_2. NON-COMP',
               'WCIO Nature of Injury Code_encoded_3. MED ONLY',
               'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
               'WCIO Nature of Injury Code_encoded_1. CANCELLED',
               'WCIO Nature of Injury Code_encoded_8. DEATH',
               'WCIO Nature of Injury Code_encoded_6. PPD NSL',
               'WCIO Nature of Injury Code_encoded_7. PTD',
               'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
               'WCIO Part Of Body Code_encoded_2. NON-COMP',
               'WCIO Part Of Body Code_encoded_3. MED ONLY',
               'WCIO Part Of Body Code_encoded_4. TEMPORARY',
               'WCIO Part Of Body Code_encoded_1. CANCELLED',
               'WCIO Part Of Body Code_encoded_8. DEATH',
               'WCIO Part Of Body Code_encoded_6. PPD NSL',
               'WCIO Part Of Body Code_encoded_7. PTD',
               'Accident Datemonth',
               'Industry Code_freq', 'WCIO Cause of Injury Code_freq',
               'WCIO Nature of Injury Code_freq', 'WCIO Part Of Body Code_freq', 
               'Carrier Type_freq', 'Carrier Name_freq']

cat_columns = ['Alternative Dispute Resolution_binary', 
               'COVID-19 Indicator_binary',
               'Attorney/Representative_binary',
               'Carrier Type_Self-insured Private Entity',
               'Carrier Type_Self-insured Public Entity', 
               'Carrier Type_Special Funds',
               'Carrier Type_State Insurance Fund', 
               'C-3 Date_nabinary',
               'First Hearing Date_nabinary', 
               'Accident Date_Season_Spring',
               'Accident Date_Season_Summer', 
               'Accident Date_Season_Winter',
               'Average Weekly Wage_nabinary']


# Create subsets
X_train_num = X_train_encoded[num_columns]
X_train_cat = X_train_encoded[cat_columns]

X_val_num = X_val_encoded[num_columns]
X_val_cat = X_val_encoded[cat_columns]

X_test_num=X_test_encoded[num_columns]
X_test_cat=X_test_encoded[cat_columns]

## 2. Data scaling

### 2.1 Normalization

> Not all models need the variables to be scaled.

In [96]:
scaler = MinMaxScaler().fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
print("Parameters fitted:")
for feature, min_val, max_val in zip(X_train_num.columns, scaler.data_min_, scaler.data_max_):
    print(f"Variable: {feature} | Min: {min_val} | Max: {max_val}")

# Convert the array to a pandas dataframe
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train_encoded.index)
X_train_num_scaled.describe().round(2)

Parameters fitted:
Variable: Age at Injury | Min: 14.0 | Max: 83.0
Variable: IME-4 Count | Min: 0.0 | Max: 20.0
Variable: Number of Dependents | Min: 0.0 | Max: 6.0
Variable: Days_between_Assembly Date_Accident Date_log | Min: 0.6931471805599453 | Max: 7.371489295214277
Variable: Days_between_C-2 Date_Imputed_Accident Date_log | Min: 0.6931471805599453 | Max: 7.510430556378006
Variable: Average Weekly Wage_Imputed_log | Min: 0.6931471805599453 | Max: 8.58443493056553
Variable: Industry Code_encoded_5. PPD SCH LOSS | Min: 0.036572848576522 | Max: 0.2038215586084509
Variable: Industry Code_encoded_2. NON-COMP | Min: 0.3642649702669063 | Max: 0.6545985401459854
Variable: Industry Code_encoded_3. MED ONLY | Min: 0.0909372579395817 | Max: 0.159809183064997
Variable: Industry Code_encoded_4. TEMPORARY | Min: 0.1543065693430657 | Max: 0.3952427050200525
Variable: Industry Code_encoded_1. CANCELLED | Min: 0.0125854468791924 | Max: 0.1379310344787534
Variable: Industry Code_encoded_8. DEATH | M

,Age at Injury,IME-4 Count,Number of Dependents,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,Average Weekly Wage_Imputed_log,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_7. PTD,WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_3. MED ONLY,WCIO Cause of Injury Code_encoded_4. TEMPORARY,WCIO Cause of Injury Code_encoded_1. CANCELLED,WCIO Cause of Injury Code_encoded_8. DEATH,WCIO Cause of Injury Code_encoded_6. PPD NSL,WCIO Cause of Injury Code_encoded_7. PTD,WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_3. MED ONLY,WCIO Nature of Injury Code_encoded_4. TEMPORARY,WCIO Nature of Injury Code_encoded_1. CANCELLED,WCIO Nature of Injury Code_encoded_8. DEATH,WCIO Nature of Injury Code_encoded_6. PPD NSL,WCIO Nature of Injury Code_encoded_7. PTD,WCIO Part Of Body Code_encoded_5. PPD SCH LOSS,WCIO Part Of Body Code_encoded_2. NON-COMP,WCIO Part Of Body Code_encoded_3. MED ONLY,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_6. PPD NSL,WCIO Part Of Body Code_encoded_7. PTD,Accident Datemonth,Industry Code_freq,WCIO Cause of Injury Code_freq,WCIO Nature of Injury Code_freq,WCIO Part Of Body Code_freq,Carrier Type_freq,Carrier Name_freq
count,394397.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00
mean,0.41,0.04,0.50,0.30,0.30,0.33,0.29,0.50,0.42,0.42,0.06,0.20,0.55,0.26,0.18,0.43,0.51,0.37,0.04,0.01,0.16,0.01,0.17,0.52,0.58,0.45,0.04,0.01,0.28,0.00,0.23,0.34,0.61,0.54,0.04,0.02,0.15,0.03,0.50,0.48,0.43,0.51,0.44,0.68,0.24
std,0.20,0.09,0.33,0.16,0.17,0.40,0.34,0.31,0.19,0.25,0.07,0.14,0.19,0.18,0.13,0.18,0.13,0.13,0.15,0.02,0.11,0.02,0.11,0.19,0.15,0.20,0.15,0.03,0.18,0.02,0.19,0.24,0.15,0.19,0.15,0.07,0.16,0.06,0.32,0.35,0.28,0.37,0.31,0.32,0.38
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.25,0.00,0.17,0.19,0.18,0.00,0.06,0.17,0.26,0.26,0.03,0.12,0.40,0.15,0.13,0.31,0.47,0.31,0.01,0.00,0.06,0.00,0.06,0.41,0.55,0.34,0.02,0.00,0.09,0.00,0.04,0.14,0.57,0.37,0.01,0.00,0.03,0.00,0.18,0.15,0.22,0.15,0.17,0.39,0.01
50%,0.41,0.00,0.50,0.27,0.26,0.00,0.12,0.54,0.44,0.45,0.04,0.15,0.63,0.27,0.17,0.38,0.54,0.38,0.02,0.00,0.16,0.01,0.19,0.53,0.60,0.44,0.02,0.00,0.26,0.00,0.22,0.25,0.61,0.58,0.02,0.00,0.10,0.01,0.55,0.38,0.37,0.36,0.38,0.43,0.05
75%,0.58,0.00,0.83,0.38,0.40,0.79,0.33,0.74,0.62,0.55,0.09,0.22,0.67,0.31,0.21,0.52,0.56,0.45,0.02,0.00,0.21,0.01,0.25,0.56,0.66,0.57,0.02,0.00,0.44,0.00,0.31,0.54,0.72,0.68,0.03,0.03,0.24,0.03,0.73,0.81,0.68,1.00,0.73,1.00,0.16
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [97]:
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val_encoded.index)
X_val_num_scaled.describe().round(2)

,Age at Injury,IME-4 Count,Number of Dependents,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,Average Weekly Wage_Imputed_log,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_7. PTD,WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_3. MED ONLY,WCIO Cause of Injury Code_encoded_4. TEMPORARY,WCIO Cause of Injury Code_encoded_1. CANCELLED,WCIO Cause of Injury Code_encoded_8. DEATH,WCIO Cause of Injury Code_encoded_6. PPD NSL,WCIO Cause of Injury Code_encoded_7. PTD,WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_3. MED ONLY,WCIO Nature of Injury Code_encoded_4. TEMPORARY,WCIO Nature of Injury Code_encoded_1. CANCELLED,WCIO Nature of Injury Code_encoded_8. DEATH,WCIO Nature of Injury Code_encoded_6. PPD NSL,WCIO Nature of Injury Code_encoded_7. PTD,WCIO Part Of Body Code_encoded_5. PPD SCH LOSS,WCIO Part Of Body Code_encoded_2. NON-COMP,WCIO Part Of Body Code_encoded_3. MED ONLY,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_6. PPD NSL,WCIO Part Of Body Code_encoded_7. PTD,Accident Datemonth,Industry Code_freq,WCIO Cause of Injury Code_freq,WCIO Nature of Injury Code_freq,WCIO Part Of Body Code_freq,Carrier Type_freq,Carrier Name_freq
count,171358.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00
mean,0.41,0.04,0.50,0.31,0.30,0.33,0.28,0.51,0.42,0.42,0.06,0.20,0.55,0.26,0.18,0.43,0.50,0.37,0.05,0.01,0.16,0.01,0.17,0.52,0.57,0.45,0.05,0.01,0.28,0.00,0.23,0.34,0.61,0.54,0.05,0.02,0.15,0.03,0.50,0.48,0.43,0.51,0.44,0.68,0.24
std,0.20,0.10,0.33,0.18,0.17,0.40,0.34,0.31,0.20,0.25,0.07,0.14,0.19,0.18,0.13,0.18,0.13,0.13,0.16,0.02,0.11,0.02,0.12,0.19,0.15,0.20,0.16,0.03,0.18,0.02,0.19,0.24,0.16,0.19,0.16,0.07,0.16,0.06,0.31,0.35,0.28,0.37,0.31,0.32,0.38
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00
25%,0.25,0.00,0.17,0.19,0.18,0.00,0.06,0.17,0.26,0.26,0.03,0.12,0.40,0.15,0.13,0.31,0.47,0.31,0.01,0.00,0.05,0.00,0.06,0.41,0.55,0.34,0.02,0.00,0.09,0.00,0.04,0.14,0.55,0.37,0.01,0.00,0.03,0.00,0.18,0.15,0.22,0.15,0.17,0.39,0.01
50%,0.41,0.00,0.50,0.27,0.26,0.00,0.12,0.54,0.44,0.45,0.04,0.15,0.63,0.27,0.17,0.38,0.54,0.38,0.02,0.00,0.16,0.01,0.19,0.53,0.60,0.44,0.02,0.00,0.26,0.00,0.22,0.26,0.61,0.58,0.02,0.00,0.10,0.01,0.55,0.38,0.37,0.36,0.38,0.43,0.05
75%,0.58,0.00,0.83,0.38,0.41,0.79,0.33,0.74,0.62,0.55,0.09,0.22,0.67,0.31,0.21,0.52,0.56,0.45,0.02,0.00,0.21,0.01,0.25,0.56,0.66,0.57,0.02,0.00,0.44,0.00,0.31,0.54,0.72,0.68,0.03,0.03,0.24,0.03,0.73,0.81,0.68,1.00,0.73,1.00,0.16
max,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [98]:
X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns).set_index(X_test_encoded.index)
X_test_num_scaled.describe().round(2)

,Age at Injury,IME-4 Count,Number of Dependents,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,Average Weekly Wage_Imputed_log,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_7. PTD,WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_3. MED ONLY,WCIO Cause of Injury Code_encoded_4. TEMPORARY,WCIO Cause of Injury Code_encoded_1. CANCELLED,WCIO Cause of Injury Code_encoded_8. DEATH,WCIO Cause of Injury Code_encoded_6. PPD NSL,WCIO Cause of Injury Code_encoded_7. PTD,WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_3. MED ONLY,WCIO Nature of Injury Code_encoded_4. TEMPORARY,WCIO Nature of Injury Code_encoded_1. CANCELLED,WCIO Nature of Injury Code_encoded_8. DEATH,WCIO Nature of Injury Code_encoded_6. PPD NSL,WCIO Nature of Injury Code_encoded_7. PTD,WCIO Part Of Body Code_encoded_5. PPD SCH LOSS,WCIO Part Of Body Code_encoded_2. NON-COMP,WCIO Part Of Body Code_encoded_3. MED ONLY,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_6. PPD NSL,WCIO Part Of Body Code_encoded_7. PTD,Accident Datemonth,Industry Code_freq,WCIO Cause of Injury Code_freq,WCIO Nature of Injury Code_freq,WCIO Part Of Body Code_freq,Carrier Type_freq,Carrier Name_freq
count,386202.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00
mean,0.40,0.01,0.50,0.29,0.28,0.15,0.26,0.54,0.43,0.40,0.06,0.19,0.54,0.25,0.18,0.44,0.51,0.36,0.04,0.00,0.16,0.01,0.17,0.53,0.58,0.43,0.04,0.01,0.28,0.00,0.23,0.34,0.63,0.54,0.04,0.02,0.15,0.02,0.43,0.46,0.42,0.53,0.48,0.70,0.21
std,0.20,0.04,0.33,0.17,0.16,0.32,0.32,0.31,0.19,0.26,0.07,0.13,0.19,0.17,0.12,0.18,0.13,0.14,0.15,0.02,0.11,0.02,0.11,0.19,0.15,0.20,0.15,0.03,0.17,0.02,0.18,0.24,0.15,0.19,0.15,0.05,0.15,0.05,0.32,0.34,0.29,0.37,0.33,0.33,0.36
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00
25%,0.23,0.00,0.17,0.16,0.16,0.00,0.06,0.17,0.26,0.26,0.03,0.08,0.40,0.15,0.13,0.31,0.48,0.31,0.01,0.00,0.06,0.00,0.06,0.41,0.56,0.34,0.02,0.00,0.17,0.00,0.05,0.12,0.57,0.37,0.01,0.00,0.03,0.00,0.18,0.15,0.21,0.15,0.19,0.39,0.01
50%,0.39,0.00,0.50,0.24,0.25,0.00,0.11,0.62,0.44,0.40,0.04,0.15,0.58,0.22,0.17,0.38,0.54,0.37,0.02,0.00,0.18,0.01,0.19,0.55,0.60,0.44,0.02,0.00,0.26,0.00,0.22,0.25,0.63,0.59,0.02,0.00,0.10,0.01,0.36,0.38,0.32,0.36,0.48,1.00,0.04
75%,0.57,0.00,0.83,0.36,0.37,0.00,0.33,0.80,0.62,0.48,0.09,0.22,0.67,0.31,0.20,0.52,0.56,0.45,0.02,0.00,0.21,0.01,0.25,0.56,0.66,0.57,0.02,0.00,0.44,0.00,0.31,0.54,0.72,0.68,0.03,0.03,0.24,0.03,0.73,0.81,0.68,1.00,0.80,1.00,0.16
max,1.00,0.90,1.00,1.05,1.02,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


### 2.2 Standarization

> We can use both, if we're using algorithms that benefit from using both.  (e.g., neural networks may benefit from normalization, while models like logistic regression may perform better with standardized data).

In [99]:
# from sklearn.preprocessing import StandardScaler

# std_scaler = StandardScaler().fit(X_train_num)
# X_train_scaled_std= scaler.transform(X_train_num)
# X_train_scaled_std = pd.DataFrame(X_train_scaled_std, columns = X_train_num.columns).set_index(X_train_encoded.index)

# X_val_scaled_std = scaler.transform(X_val_num)
# X_val_scaled_std = pd.DataFrame(X_val_scaled_std, columns = X_val_num.columns).set_index(X_val_encoded.index)


## 3. Feature Selection

### 3.1 Categorical: Filter Methods

#### 3.1.1 Chi square and Cramer's V

In [100]:
def cramers_v(X, y):
    """
    Calculate Cramér's V for a categorical feature and a target.
    :param X: Categorical feature (Pandas Series or array-like)
    :param y: Target variable (Pandas Series or array-like)
    :return: Cramér's V value
    """
    # Create the contingency table
    df_contingency = pd.crosstab(X, y)

    # Perform chi-square test
    chi2, p, dof, expected = chi2_contingency(df_contingency.values)

    # Calculate Cramér's V
    n = df_contingency.sum().sum()  # Total number of observations
    min_dim = min(df_contingency.shape) - 1  # Min between number of rows and columns - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula

    return cramers_v


In [101]:
# Perform Chi-square test
chi2_values, p_values = chi2(X_train_cat, y)

# Create DataFrame for Chi-square results
chi2_results = pd.DataFrame({
    'Column': X_train_cat.columns,
    'Chi2': chi2_values.round(5),
    'p-value': p_values.round(5)
})

# Calculate Cramér's V for binary target
cramers_v_values = []
for var in X_train_cat.columns:
    v = cramers_v(X_train_cat[var], y)
    cramers_v_values.append(v)

# Add Cramér's V to DataFrame
chi2_results['Cramér\'s V'] = cramers_v_values

# Filter important features
chi2_important_features = chi2_results[(chi2_results['p-value'] < 0.05) & (chi2_results['Cramér\'s V'] >= 0.1)]

list_features_chi2_cramer = chi2_important_features['Column'].values

print(chi2_important_features)

                                     Column          Chi2  p-value  Cramér's V
2            Attorney/Representative_binary  102697.45907      0.0    0.614944
4   Carrier Type_Self-insured Public Entity    5848.47449      0.0    0.137122
6         Carrier Type_State Insurance Fund    4653.20236      0.0    0.120764
7                         C-3 Date_nabinary   29753.82707      0.0    0.479025
8               First Hearing Date_nabinary   30991.88712      0.0    0.545735
12             Average Weekly Wage_nabinary  110326.13914      0.0    0.872155


#### 3.1.2 Mutual Information

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.preprocessing import LabelEncoder

# Assuming X_train_cat is your categorical feature set and y_train is the target
# For classification problems:
mi = mutual_info_classif(X_train_cat, y,discrete_features=True)

# Create a DataFrame with the results
mi_results = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Mutual Information': mi
})

# Sort the features by their mutual information score
mi_results = mi_results.sort_values(by='Mutual Information', ascending=False)

# Display the top features based on MI
print("Mutual Information Results:")
mi_results


In [15]:
mi_important_features = mi_results[mi_results['Mutual Information'] > 0.01]

list_features_mi = mi_results[mi_results['Mutual Information'] > 0.01]['Feature'].values

print(mi_important_features)

                           Feature  Mutual Information
12    Average Weekly Wage_nabinary            0.469901
2   Attorney/Representative_binary            0.202553
8      First Hearing Date_nabinary            0.159525
7                C-3 Date_nabinary            0.117598


> For the categorical we should look into keeping with:
1. Chi test: p-value < 0.05
2. Cramer's V > 0.1
3. Mutual information: above 0.1

#### 3.1.3 Drop categorical

In [22]:
categoricals_to_keep = list(set(list_features_chi2_cramer) | set(list_features_mi))


['Carrier Type_State Insurance Fund',
 'First Hearing Date_nabinary',
 'C-3 Date_nabinary',
 'Average Weekly Wage_nabinary',
 'Carrier Type_Self-insured Public Entity',
 'Attorney/Representative_binary']

In [23]:
# Drop features
X_train_cat_filtered = X_train_cat[categoricals_to_keep]
X_val_cat_filtered = X_val_cat[categoricals_to_keep]
X_test_cat_filtered = X_test_cat[categoricals_to_keep]

# Verify the remaining columns
print("Remaining features in X_train_cat:", X_train_cat_filtered.columns.tolist())


Remaining features in X_train_cat: ['Carrier Type_State Insurance Fund', 'First Hearing Date_nabinary', 'C-3 Date_nabinary', 'Average Weekly Wage_nabinary', 'Carrier Type_Self-insured Public Entity', 'Attorney/Representative_binary']


### 3.2 Numerical: Filter Methods

#### 3.2.1 Univariate

In [24]:
X_train_num_scaled.var().sort_values(ascending=False)

Average Weekly Wage_Imputed_log                       0.158004
Carrier Name_freq                                     0.142667
WCIO Nature of Injury Code_freq                       0.136735
Industry Code_freq                                    0.122980
Industry Code_encoded_5. PPD SCH LOSS                 0.114378
Number of Dependents                                  0.111300
Carrier Type_freq                                     0.104297
Accident Datemonth                                    0.099293
WCIO Part Of Body Code_freq                           0.098325
Industry Code_encoded_2. NON-COMP                     0.096515
WCIO Cause of Injury Code_freq                        0.080853
Industry Code_encoded_4. TEMPORARY                    0.064312
WCIO Part Of Body Code_encoded_2. NON-COMP            0.057635
WCIO Nature of Injury Code_encoded_4. TEMPORARY       0.039304
Age at Injury                                         0.039172
Industry Code_encoded_3. MED ONLY                     0

#### 3.2.2 Spearman Correlation

Spearman correlation because nonlinear and encoded

In [27]:
# Calculate Spearman correlation matrix
cor_spearman = X_train_num_scaled.corr(method='spearman')

#  Flatten the correlation matrix and reset the index
correlation_pairs = cor_spearman.unstack().reset_index()

# Rename columns for clarity
correlation_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# Filter the table for correlations > 0.5 or < -0.5 and exclude self-correlations (diagonal)
strong_correlations = correlation_pairs[
    ((correlation_pairs['Correlation'] > 0.8) | (correlation_pairs['Correlation'] < -0.8)) &
    (correlation_pairs['Feature_1'] != correlation_pairs['Feature_2'])
]

# Drop duplicate pairs (keep unique pairs only)
strong_correlations = strong_correlations.sort_values(by='Correlation', ascending=False).drop_duplicates(subset=['Correlation'])

# Display the table
strong_correlations


,Feature_1,Feature_2,Correlation
139,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,0.954048
1521,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_6. PPD NSL,0.811993
692,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_4. TEMPORARY,-0.820819
277,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,-0.834576
1060,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_4. TEMPORARY,-0.878413


#### 3.2.3 Mutual Information

In [28]:
# from sklearn.feature_selection import mutual_info_classif

# # Compute mutual information between categorical features and target
# mutual_info = mutual_info_classif(X_train_num_scaled, y_train_encoded)

# # Display features sorted by mutual information
# mi_results = pd.DataFrame({
#     'Feature': X_train_num_scaled.columns,
#     'Mutual Information': mutual_info
# }).sort_values(by='Mutual Information', ascending=False)

# mi_results

ValueError: Input X contains NaN.

#### 3.2.4 Drop numerical

In [ ]:
# List of features to drop from X_train_num_scaled
num_features_to_drop = [
    'Days_between_C-2 Date_Imputed_Accident Date_log'
]

# Drop features
X_train_num_scaled_filtered = X_train_num_scaled.drop(columns=num_features_to_drop)
X_val_num_scaled_filtered = X_val_num_scaled.drop(columns=num_features_to_drop)
X_test_num_scaled_filtered = X_test_num_scaled.drop(columns=num_features_to_drop)

# Verify the remaining columns
print("Remaining features in X_train_num_scaled:", X_train_num_scaled_filtered.columns.tolist())


Remaining features in X_train_num_scaled: ['Age at Injury', 'IME-4 Count', 'Number of Dependents', 'Days_between_Assembly Date_Accident Date_log', 'Average Weekly Wage_Imputed_log', 'Industry Code_encoded_5. PPD SCH LOSS', 'Industry Code_encoded_2. NON-COMP', 'Industry Code_encoded_3. MED ONLY', 'Industry Code_encoded_4. TEMPORARY', 'Industry Code_encoded_1. CANCELLED', 'Industry Code_encoded_8. DEATH', 'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD', 'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS', 'WCIO Cause of Injury Code_encoded_2. NON-COMP', 'WCIO Cause of Injury Code_encoded_3. MED ONLY', 'WCIO Cause of Injury Code_encoded_4. TEMPORARY', 'WCIO Cause of Injury Code_encoded_1. CANCELLED', 'WCIO Cause of Injury Code_encoded_8. DEATH', 'WCIO Cause of Injury Code_encoded_6. PPD NSL', 'WCIO Cause of Injury Code_encoded_7. PTD', 'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS', 'WCIO Nature of Injury Code_encoded_2. NON-COMP', 'WCIO Nature of Injury Code_enco

### 3.3 Combine the filtered datasets

In [ ]:
# Combine the filtered datasets
X_train = pd.concat([X_train_cat_filtered, X_train_num_scaled_filtered], axis=1)
X_val =  pd.concat([X_val_cat_filtered, X_val_num_scaled_filtered], axis=1)
X_test =  pd.concat([X_test_cat_filtered, X_test_num_scaled_filtered], axis=1)


# Verify the shape of the combined dataset
print("Shape of combined X_train:", X_train.shape)


Shape of combined X_train: (396097, 50)


In [71]:
X_train.drop(columns=['Age at Injury'], inplace=True)
X_val.drop(columns=['Age at Injury'], inplace=True)
X_test.drop(columns=['Age at Injury'], inplace=True)


KeyError: "['Age at Injury'] not found in axis"

### 3.4 Random forest, LASSO, Ridge

#### 3.4.1 Random Forest

In [32]:
# from sklearn.ensemble import RandomForestClassifier

# # Initialize the Random Forest Classifier
# rf_model = RandomForestClassifier(n_estimators=100, random_state=47)

# # Fit the model on X_train and y_train_encoded
# rf_model.fit(X_train, y_train_encoded)

# # Get feature importances
# importances = rf_model.feature_importances_

# # Sort feature importances in descending order
# important_indices = importances.argsort()[::-1]

# # Print the feature importances
# print("Feature importances:", importances)
# print("Sorted feature indices:", important_indices)


Feature importances: [0.00337961 0.03291874 0.01969955 0.17200544 0.00372861 0.03877776
 0.04714093 0.04443685 0.0280961  0.04713261 0.15102057 0.0111614
 0.00761336 0.01034212 0.008061   0.00724666 0.00742453 0.00729057
 0.00706422 0.01291412 0.01248329 0.01250745 0.0133258  0.01237083
 0.00918736 0.0116248  0.00866942 0.00915604 0.01046823 0.00774539
 0.00953152 0.00671216 0.0047133  0.00706006 0.00469981 0.0141976
 0.01053211 0.01197388 0.01129389 0.0102198  0.007643   0.01048357
 0.00733745 0.03373693 0.00756341 0.01088939 0.00507762 0.0092471
 0.00819221 0.03390184]
Sorted feature indices: [ 3 10  6  9  7  5 49 43  1  8  2 35 22 19 21 20 23 37 25 38 11 45 36 41
 28 13 39 30 47 24 27 26 48 14 29 40 12 44 16 42 17 15 18 33 31 46 32 34
  4  0]


In [33]:
# from xgboost import XGBClassifier
# import numpy as np

# # Initialize the XGBoost Classifier
# xgb_model = XGBClassifier(n_estimators=100, random_state=47, use_label_encoder=False, eval_metric="mlogloss")

# # Fit the model on X_train and y_train_encoded
# xgb_model.fit(X_train, y_train_encoded)

# # Get feature importances
# importances = xgb_model.feature_importances_

# # Sort feature importances in descending order
# important_indices = np.argsort(importances)[::-1]

# # Print the feature importances
# print("Feature importances:", importances)
# print("Sorted feature indices:", important_indices)


c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:18:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Feature importances: [0.00512536 0.01675951 0.00648811 0.6287525  0.01012589 0.0949544
 0.00162426 0.01384911 0.00098001 0.00269364 0.04928437 0.01369486
 0.00225702 0.00234057 0.00276251 0.00276226 0.00173098 0.0027468
 0.00172367 0.00397403 0.00166947 0.01828355 0.00185888 0.0052832
 0.00164824 0.00150402 0.00144209 0.00439345 0.00612926 0.00833023
 0.00732071 0.01126072 0.00238731 0.00163747 0.00143903 0.02151078
 0.00199086 0.00642436 0.00235488 0.00190751 0.0030102  0.00371342
 0.00123978 0.00132614 0.00203738 0.00135507 0.00192116 0.00133721
 0.00705651 0.00359711]
Sorted feature indices: [ 3  5 10 35 21  1  7 11 31  4 29 30 48  2 37 28 23  0 27 19 41 49 40 14
 15 17  9 32 38 13 12 44 36 46 39 22 16 18 20 24 33  6 25 26 34 45 47 43
 42  8]


In [34]:
# import pandas as pd
# from xgboost import XGBClassifier

# # Initialize the XGBoost Classifier
# xgb_model = XGBClassifier(n_estimators=100, random_state=47, use_label_encoder=False, eval_metric="mlogloss")

# # Fit the model on X_train and y_train_encoded
# xgb_model.fit(X_train, y_train_encoded)

# # Get feature importances
# importances = xgb_model.feature_importances_

# # Create a DataFrame with feature names and their importances
# feature_importance_df = pd.DataFrame({
#     'Feature': X_train.columns,  # Assumes X_train is a pandas DataFrame
#     'Importance': importances
# })

# # Sort the DataFrame by importance in descending order
# feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# # Print the sorted feature names and their importance
# print(feature_importance_df)


c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


                                              Feature  Importance
3                        Average Weekly Wage_nabinary    0.628753
5                      Attorney/Representative_binary    0.094954
10                    Average Weekly Wage_Imputed_log    0.049284
35     WCIO Part Of Body Code_encoded_5. PPD SCH LOSS    0.021511
21      WCIO Cause of Injury Code_encoded_3. MED ONLY    0.018284
1                         First Hearing Date_nabinary    0.016760
7                                         IME-4 Count    0.013849
11              Industry Code_encoded_5. PPD SCH LOSS    0.013695
31    WCIO Nature of Injury Code_encoded_1. CANCELLED    0.011261
4             Carrier Type_Self-insured Public Entity    0.010126
29     WCIO Nature of Injury Code_encoded_3. MED ONLY    0.008330
30    WCIO Nature of Injury Code_encoded_4. TEMPORARY    0.007321
48                                  Carrier Type_freq    0.007057
2                                   C-3 Date_nabinary    0.006488
37        

In [35]:
# import numpy as np

# # Calculate cumulative sum of feature importances
# cumulative_importance = np.cumsum(importances[important_indices])

# # Find the index where cumulative importance exceeds 90%
# threshold = 0.90
# index_90 = np.argmax(cumulative_importance >= threshold)

# # The number of features that explain 90% of the importance
# num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

# print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# # Select the top features based on this number
# random_forest_features = X_train.columns[important_indices[:num_features_90]]
# print(f"Selected features that explain 90% of cumulative importance: {random_forest_features}")


Number of features explaining 90% of cumulative importance: 13
Selected features that explain 90% of cumulative importance: Index(['Average Weekly Wage_nabinary', 'Attorney/Representative_binary',
       'Average Weekly Wage_Imputed_log',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'First Hearing Date_nabinary', 'IME-4 Count',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_1. CANCELLED',
       'Carrier Type_Self-insured Public Entity',
       'WCIO Nature of Injury Code_encoded_3. MED ONLY',
       'WCIO Nature of Injury Code_encoded_4. TEMPORARY', 'Carrier Type_freq'],
      dtype='object')


In [36]:
# import numpy as np
# import pandas as pd

# # Calculate cumulative sum of feature importances
# cumulative_importance = np.cumsum(feature_importance_df['Importance'])

# # Find the index where cumulative importance exceeds 90%
# threshold = 0.90
# index_90 = np.argmax(cumulative_importance >= threshold)

# # The number of features that explain 90% of the importance
# num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

# print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# # Select the top features based on this number
# xgboost_top_features = feature_importance_df['Feature'].iloc[:num_features_90]
# print(f"Selected features that explain 90% of cumulative importance:\n{xgboost_top_features}")


Number of features explaining 90% of cumulative importance: 13
Selected features that explain 90% of cumulative importance:
3                        Average Weekly Wage_nabinary
5                      Attorney/Representative_binary
10                    Average Weekly Wage_Imputed_log
35     WCIO Part Of Body Code_encoded_5. PPD SCH LOSS
21      WCIO Cause of Injury Code_encoded_3. MED ONLY
1                         First Hearing Date_nabinary
7                                         IME-4 Count
11              Industry Code_encoded_5. PPD SCH LOSS
31    WCIO Nature of Injury Code_encoded_1. CANCELLED
4             Carrier Type_Self-insured Public Entity
29     WCIO Nature of Injury Code_encoded_3. MED ONLY
30    WCIO Nature of Injury Code_encoded_4. TEMPORARY
48                                  Carrier Type_freq
Name: Feature, dtype: object


#### 3.4.2 Lasso

In [39]:
# # Check which columns contain missing values
# missing_columns = X_train.columns[X_train.isnull().any()]
# print(missing_columns)


Index(['Age at Injury'], dtype='object')


In [42]:
# from sklearn.linear_model import LassoCV
# lasso = LassoCV(cv=5)  # Cross-validation to select optimal alpha
# lasso.fit(X_train, y_train_encoded)
# lasso_features = X_train.columns[lasso.coef_ != 0]
# print("Selected features by Lasso:", lasso_features)

Selected features by Lasso: Index(['First Hearing Date_nabinary', 'C-3 Date_nabinary',
       'Average Weekly Wage_nabinary',
       'Carrier Type_Self-insured Public Entity',
       'Attorney/Representative_binary', 'IME-4 Count',
       'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage_Imputed_log',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_2. NON-COMP',
       'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
       'WCIO Nature of Injury Code_encoded_6. PPD NSL',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Part Of Body Code_encoded_2. NON-COMP',
       'WCIO Part Of 

#### 3.4.3 Ridge

In [ ]:
# from sklearn.linear_model import RidgeCV
# import numpy as np

# # Initialize Ridge regression model with cross-validation
# ridge_model = RidgeCV(cv=5)

# # Fit the model on the training data
# ridge_model.fit(X_train, y_train_encoded)

# # Get the feature coefficients
# coefficients = ridge_model.coef_

# # Set a threshold to select features (e.g., absolute coefficient > 0.01)
# threshold = 0.01
# ridge_features = X_train.columns[np.abs(coefficients) > threshold]

# print(f"Selected features using Ridge regression: {ridge_features}")


Selected features using Ridge regression: Index(['Carrier Type_State Insurance Fund', 'First Hearing Date_nabinary',
       'C-3 Date_nabinary', 'Average Weekly Wage_nabinary',
       'Carrier Type_Self-insured Public Entity',
       'Attorney/Representative_binary', 'IME-4 Count',
       'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage_Imputed_log',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_2. NON-COMP',
       'Industry Code_encoded_3. MED ONLY',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_1. CANCELLED',
       'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_2. NON-COMP',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code

#### 3.4.4 Combined results

> VOTING: AT LEAST 2 OUT OF 3

In [55]:
# # Find features selected by exactly two out of the three methods
# final_features_set = (
#     (rf_set & lasso_set) - (rf_set & lasso_set & ridge_set) |
#     (rf_set & ridge_set) - (rf_set & lasso_set & ridge_set) |
#     (lasso_set & ridge_set) - (rf_set & lasso_set & ridge_set)
# )

# # Convert to sorted list
# final_features = sorted(final_features_set)

# # Display the final feature set
# print(f"Features selected by exactly 2 methods ({len(final_features)} features):")
# print(final_features)


Features selected by exactly 2 methods (19 features):
['Accident Datemonth', 'C-3 Date_nabinary', 'Carrier Name_freq', 'Days_between_Assembly Date_Accident Date_log', 'Industry Code_encoded_4. TEMPORARY', 'Industry Code_encoded_6. PPD NSL', 'Industry Code_freq', 'WCIO Cause of Injury Code_encoded_1. CANCELLED', 'WCIO Cause of Injury Code_encoded_3. MED ONLY', 'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS', 'WCIO Cause of Injury Code_encoded_6. PPD NSL', 'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_encoded_1. CANCELLED', 'WCIO Nature of Injury Code_encoded_2. NON-COMP', 'WCIO Nature of Injury Code_encoded_3. MED ONLY', 'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS', 'WCIO Nature of Injury Code_encoded_6. PPD NSL', 'WCIO Part Of Body Code_encoded_2. NON-COMP', 'WCIO Part Of Body Code_freq']


### 3.5 Final features

In [76]:
variaveis = ['Attorney/Representative_binary',
             'Age at Injury',
             'Average Weekly Wage_Imputed_log',
             'C-3 Date_nabinary',             
             'Carrier Type_Self-insured Public Entity',
             'Carrier Type_State Insurance Fund',
             'Days_between_Assembly Date_Accident Date_log',
             'First Hearing Date_nabinary',
             'IME-4 Count',
            
             'Industry Code_encoded_8. DEATH',
             'Industry Code_encoded_6. PPD NSL',

             'WCIO Cause of Injury Code_encoded_1. CANCELLED',
             'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
             'WCIO Cause of Injury Code_encoded_6. PPD NSL',
             'WCIO Cause of Injury Code_encoded_7. PTD',
             'WCIO Cause of Injury Code_encoded_8. DEATH',

             'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
             'WCIO Nature of Injury Code_encoded_6. PPD NSL',
             
             'WCIO Part Of Body Code_encoded_1. CANCELLED',
             'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
             'WCIO Part Of Body Code_encoded_6. PPD NSL',
             'WCIO Cause of Injury Code_encoded_3. MED ONLY',
             'WCIO Part Of Body Code_encoded_3. MED ONLY',

             
             'WCIO Nature of Injury Code_encoded_3. MED ONLY',
             'Industry Code_encoded_3. MED ONLY',
             'Industry Code_encoded_4. TEMPORARY',
             'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
             'WCIO Part Of Body Code_encoded_4. TEMPORARY'
             
             
             
             ]

In [77]:
X_train = X_train[variaveis]
X_val = X_val[variaveis]
X_test = X_test[variaveis]

## SMOTE

In [ ]:
# Assuming `y` is your target variable
class_distribution = y_train_encoded.value_counts()
class_percentages = y_train_encoded.value_counts(normalize=True) * 100

# Combine into a single DataFrame for better visualization
distribution_df = pd.DataFrame({
    'Class': class_distribution.index,
    'Count': class_distribution.values,
    'Percentage (%)': class_percentages.values
})
print(distribution_df)


   Class   Count  Percentage (%)
0      1  202245       51.059463
1      3  101242       25.559901
2      2   47591       12.014986
3      4   33513        8.460806
4      0    8191        2.067928
5      5    2924        0.738203
6      7     324        0.081798
7      6      67        0.016915


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler



# Define undersampling strategy based on the actual class distribution
undersample_strategy = {
    1: 20224,
    3: 20248,
    2: 47591,
    4: 33513,
    0: 8191,
    5: 2924,
    7: 324,
    6:67
}

# Apply undersampling to majority classes
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train_encoded)

# # Define SMOTE strategy for oversampling the minority classes
# smote = SMOTE(sampling_strategy={
#     7:807,

#     6: 670,   # Oversample class '5' to 3,000
# }, random_state=42)

# # Apply SMOTE to oversample minority classes
# X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

# Check the new class distribution after resampling
unique_classes_final, class_counts_final = np.unique(y_resampled, return_counts=True)
class_distribution_final = dict(zip(unique_classes_final, class_counts_final))
print("New Class Distribution after Resampling:", class_distribution_final)

New Class Distribution after Resampling: {0: 8191, 1: 20224, 2: 47591, 3: 20248, 4: 33513, 5: 2924, 6: 67, 7: 324}


In [80]:
weights = np.ones(len(y_resampled))
weights

majority_class_label_1 = 1  # Replace with your actual majority class label
majority_class_label_3 = 3



weights[y_resampled == majority_class_label_1] = 10
weights[y_resampled == majority_class_label_3] = 5
weights

array([1., 1., 1., ..., 1., 1., 1.])

In [81]:
import xgboost as xgb

model = xgb.XGBClassifier(eval_metric="mlogloss")
model.fit(X_resampled, y_resampled, sample_weight=weights)

# 3. Make predictions
y_pred = model.predict(X_val)  # Assuming X_test is available

In [82]:

# Evaluate the model
accuracy = accuracy_score(y_val_encoded, y_pred)
f1_macro = f1_score(y_val_encoded, y_pred, average='macro')  # Macro-averaged F1 score

# Print individual model results
print(f"XGBClassifier Accuracy: {accuracy:.4f}")
print(f"XGBClassifier Macro F1 Score: {f1_macro:.4f}")
print(classification_report(y_val_encoded, y_pred))  # Detailed report including precision, recall, and F1 score per class
print("-" * 50)

# Display the summary of results
print("\nModel Evaluation Results Summary:")
print(f"XGBClassifier: Accuracy = {accuracy:.4f}, Macro F1 Score = {f1_macro:.4f}")


XGBClassifier Accuracy: 0.7748
XGBClassifier Macro F1 Score: 0.4307
              precision    recall  f1-score   support

           0       0.55      0.49      0.52      3741
           1       0.85      0.96      0.90     87319
           2       0.39      0.11      0.17     20672
           3       0.73      0.83      0.77     44551
           4       0.64      0.62      0.63     14484
           5       0.09      0.00      0.01      1263
           6       0.00      0.00      0.00        29
           7       0.47      0.42      0.44       141

    accuracy                           0.77    172200
   macro avg       0.46      0.43      0.43    172200
weighted avg       0.73      0.77      0.74    172200

--------------------------------------------------

Model Evaluation Results Summary:
XGBClassifier: Accuracy = 0.7748, Macro F1 Score = 0.4307


## APPLY TO TEST

In [83]:
y_test_pred = model.predict(X_test)

In [84]:
# Analyze the distribution of predicted classes
predictions_df = pd.DataFrame(y_test_pred, columns=["Predicted Label"])
print("Predicted Class Distribution:")
print(predictions_df["Predicted Label"].value_counts(normalize=True))


Predicted Class Distribution:
Predicted Label
1    0.797930
3    0.126201
2    0.036227
4    0.019759
0    0.019527
7    0.000325
5    0.000031
Name: proportion, dtype: float64


In [85]:
X_previsao = X_test.copy()

In [86]:
X_previsao.loc[:, 'Predicted_Claim_Injury_Type'] = y_test_pred

In [87]:
claim_injury_type_mapping = {
    '4. TEMPORARY': 3,
    '2. NON-COMP': 1,
    '5. PPD SCH LOSS': 4,
    '3. MED ONLY': 2,
    '6. PPD NSL': 5,
    '1. CANCELLED': 0,
    '8. DEATH': 7,
    '7. PTD': 6
}

# Reverse the mapping
reverse_claim_injury_type_mapping = {v: k for k, v in claim_injury_type_mapping.items()}

# Use the reversed mapping to get the original labels
X_previsao['Predicted_Claim_Injury_Type'] = X_previsao['Predicted_Claim_Injury_Type'].map(reverse_claim_injury_type_mapping)


In [88]:
X_previsao[['Predicted_Claim_Injury_Type']].to_csv('test_predictions.csv')


## 4. Model building

> Não está em 1-D o y

In [89]:
# y_train = y_train_encoded
# y_val = y_val_encoded

In [90]:
# from sklearn.metrics import accuracy_score, classification_report, f1_score
# from sklearn.model_selection import train_test_split

# # Define the models
# models = {
#     "MLPClassifier": MLPClassifier(max_iter=500),
#     "RandomForest": RandomForestClassifier(),
#     "LogisticRegression": LogisticRegression(max_iter=500),
#     "KNeighborsClassifier": KNeighborsClassifier(),
#     "DecisionTree": DecisionTreeClassifier(),
#     "GaussianNB": GaussianNB(),
#     "RidgeClassifier": RidgeClassifier()
# }

# # Dictionary to store evaluation results
# results = {}

# # Iterate over each model
# for model_name, model in models.items():
#     print(f"Training {model_name}...")
#     # Fit the model on the training data
#     model.fit(X_train, y_train)
    
#     # Predict on the validation data
#     y_pred = model.predict(X_val)
    
#     # Evaluate the model
#     accuracy = accuracy_score(y_val, y_pred)
#     f1_macro = f1_score(y_val, y_pred, average='macro')  # Macro-averaged F1 score
    
#     # Store results
#     results[model_name] = {
#         "accuracy": accuracy,
#         "f1_macro": f1_macro
#     }
    
#     # Print individual model results
#     print(f"{model_name} Accuracy: {accuracy:.4f}")
#     print(f"{model_name} Macro F1 Score: {f1_macro:.4f}")
#     print(classification_report(y_val, y_pred))  # Detailed report including precision, recall, and F1 score per class
#     print("-" * 50)

# # Display the summary of results
# print("\nModel Evaluation Results Summary:")
# for model_name, metrics in results.items():
#     print(f"{model_name}: Accuracy = {metrics['accuracy']:.4f}, Macro F1 Score = {metrics['f1_macro']:.4f}")
